In [1]:
import pandas as pd
import unicodedata
import re
from thefuzz import process
from typing import List
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.options.display.max_columns = 111
pd.options.display.max_rows = 20

In [3]:
def to_string(x):
    return str(x)

In [4]:
from deep_translator import GoogleTranslator

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    if only_ascii.decode('utf-8') == ' ':
        print("Translating: " + input_str)
        nfkd_form = unicodedata.normalize('NFKD', GoogleTranslator(source='auto', target='en').translate(input_str))
        only_ascii = nfkd_form.encode('ASCII', 'ignore')
        return only_ascii.decode('utf-8')
    else:
        return only_ascii.decode('utf-8')

In [5]:
def remove_alphanumeric(input_str):
    return re.sub(r'\W+', '', input_str).lower()

In [6]:
def best_matching_name(name, name_list):
    result = process.extractOne(name, name_list, score_cutoff=80)
    return result[0] if result else None

In [7]:
player_stats = pd.read_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\player_stats.csv")

In [9]:
transfers = pd.read_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\transfers.csv")
transfers = transfers[transfers["Aufnehmende Liga"].isin(["Premier League","Bundesliga","LaLiga","Ligue 1","Serie A"])]
transfers = transfers[transfers["Saison"] <= 23]

In [10]:
fifa = pd.read_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\fifa_stats.csv")

In [11]:
fifa.drop(columns=['player_url', 'fifa_update', 'update_as_of',
 'player_positions', 'value_eur', 'wage_eur', 'age', 'dob', 'club_team_id', 'league_id','league_name', 'league_level',
 'club_position', 'club_jersey_number', 'club_loaned_from',
 'club_joined_date', 'club_contract_valid_until_year', 'nationality_id',
 'nationality_name', 'nation_team_id', 'nation_position',
 'nation_jersey_number', 'real_face', 'release_clause_eur', 'player_tags',
 'player_traits', 'ls', 'st','rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb',
 'rcb', 'rb', 'gk'],
  axis=1, inplace=True)

In [12]:
fifa["fifa_version"] = fifa["fifa_version"] - 1

In [13]:
fifa = fifa[fifa["fifa_version"] >= 18]

In [14]:
fifa["fifa_version"] = fifa["fifa_version"].apply(lambda x: int(x))

In [15]:
fifa["club_name"] = fifa["club_name"].apply(to_string)

Correcting values

In [16]:
fifa.rename(columns={"fifa_version":"Saison"},inplace=True)
fifa["fifa_name"] = fifa["long_name"].apply(remove_accents)
fifa["fifa_name"] = fifa["fifa_name"].apply(remove_alphanumeric)
fifa["fifa_club"] = fifa["club_name"].apply(remove_accents)
fifa["fifa_club"] = fifa["fifa_club"].apply(remove_alphanumeric)

Translating: 손흥민 孙兴慜
Translating: 김민재 金敏在
Translating: 鎌田 大地
Translating: 久保 建英
Translating: 三笘 薫
Translating: 遠藤 航
Translating: 冨安 健洋
Translating: 旗手 怜央
Translating: 堂安 律
Translating: 이재성 李在成
Translating: 伊东 纯也
Translating: 守田 英正
Translating: 板仓 晃
Translating: 古橋 亨梧
Translating: 中村 航輔
Translating: 菅原 由勢
Translating: 황희찬 黄喜灿
Translating: 中村 敬斗
Translating: 伊藤 宏樹
Translating: 本 寛也
Translating: 윤빛가람 尹光河
Translating: 조현우 崔永宇
Translating: 주민규 朱文奎
Translating: 南野 拓実
Translating: 浅野 佐藤
Translating: 나상호 那尚昊
Translating: 奥川 雅也
Translating: 前田 大然
Translating: 上田 綺世
Translating: 髙丘 陽平
Translating: 기성용 寄诚庸
Translating: 長谷 部誠
Translating: 吉田 麻也
Translating: 김영권 金英权
Translating: 田中 青
Translating: 岩田 智輝
Translating: 相馬 勇紀
Translating: 坂元 達裕
Translating: 김승규 金承奎
Translating: 이창근 李灿东
Translating: 임채민 林查明
Translating: 原口 元気
Translating: 안현범 安铉范
Translating: 森岡 弘良太
Translating: 고승범 高承范
Translating: 斉藤 光毅
Translating: 町田 浩樹
Translating: 三竿 健斗
Translating: 김대원 金大元
Translating: 홍정호 洪正好
Translating: 신진호 申振豪

Lade schon übersetze fifa datei ein

In [17]:
fifa["Saison"] = fifa["Saison"].apply(lambda x: str(x))

In [18]:
player_stats.rename(columns={"Verein":"Abgebender Verein"},inplace=True)
player_stats["Saison"] = player_stats["Saison"].apply(lambda x: str(x+1))
player_stats["stats_name"] = player_stats["Spieler"].apply(remove_accents)
player_stats["stats_name"] = player_stats["stats_name"].apply(remove_alphanumeric)
player_stats["stats_club"] = player_stats["Abgebender Verein"].apply(remove_accents)
player_stats["stats_club"] = player_stats["stats_club"].apply(remove_alphanumeric)

In [19]:
transfers["Saison"] = transfers["Saison"].apply(to_string)
transfers["ID"] = transfers["Spieler"].apply(remove_accents)
transfers["ID"] = transfers["ID"].apply(remove_alphanumeric)
transfers["clubID"] = transfers["Abgebender Verein"].apply(remove_accents)
transfers["clubID"] = transfers["clubID"].apply(remove_alphanumeric)

Constructing a matching names dataset (only needed once, then stored as csv)

In [20]:
stats_names = list(set(player_stats["stats_name"].values.tolist()))

In [21]:
matching_names = pd.DataFrame()
matching_names["ID"] = transfers["Spieler"].apply(remove_accents).apply(remove_alphanumeric)
matching_names['stats_name'] = matching_names["ID"].apply(lambda x: best_matching_name(x, stats_names))

In [22]:
matching_names = matching_names.drop_duplicates()

In [23]:
matching_names.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_names.csv",index=False)

In [24]:
stats_clubs = list(set(player_stats["stats_club"].values.tolist()))

In [25]:
matching_clubs = pd.DataFrame()
matching_clubs["clubID"] = transfers["Abgebender Verein"].apply(remove_accents).apply(remove_alphanumeric)
matching_clubs["stats_club"] = matching_clubs["clubID"].apply(lambda x: best_matching_name(x, stats_clubs))

In [26]:
matching_clubs = matching_clubs.drop_duplicates()

In [27]:
matching_clubs.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_clubs.csv",index=False)

Dasselbe für fifa datensatz

In [28]:
fifa_names = list(set(fifa["fifa_name"].values.tolist()))

In [29]:
matching_names_fifa = pd.DataFrame()
matching_names_fifa["ID"] = transfers["Spieler"].apply(remove_accents).apply(remove_alphanumeric)
matching_names_fifa['fifa_name'] = matching_names_fifa["ID"].apply(lambda x: best_matching_name(x, fifa_names))

In [30]:
matching_names_fifa = matching_names_fifa.drop_duplicates()

In [31]:
matching_names_fifa.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_names_fifa.csv",index=False)

In [32]:
fifa_clubs = list(set(fifa["fifa_club"].values.tolist()))

In [33]:
matching_clubs_fifa = pd.DataFrame()
matching_clubs_fifa["clubID"] = transfers["Abgebender Verein"].apply(remove_accents).apply(remove_alphanumeric)
matching_clubs_fifa["fifa_club"] = matching_clubs_fifa["clubID"].apply(lambda x: best_matching_name(x, fifa_clubs))

In [34]:
matching_clubs_fifa = matching_clubs_fifa.drop_duplicates()

In [35]:
matching_clubs_fifa.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\raw_data\\matching_clubs_fifa.csv",index=False)

Merging

In [36]:
transfers_merged = pd.merge(transfers,matching_names,on=["ID"])

In [37]:
transfers_merged = pd.merge(transfers_merged,matching_clubs,on=["clubID"])

In [38]:
transfers_merged = pd.merge(transfers_merged,matching_names_fifa,on=["ID"])

In [39]:
transfers_merged = pd.merge(transfers_merged,matching_clubs_fifa,on=["clubID"])

In [40]:
transfers_merged = pd.merge(transfers_merged,player_stats,on=["stats_name","stats_club","Saison"])

In [41]:
transfers_merged = pd.merge(transfers_merged,fifa,on=["fifa_name","Saison"])

In [42]:
transfers

,Saison,Spieler,Position,Alter,Marktwert,Nationalität,Abgebender Verein,Abgebende Liga,Aufnehmender Verein,Aufnehmende Liga,Ablöse,ID,clubID
0,17,Neymar,Left Winger,25,100.0,Brazil,Barcelona,LaLiga,Paris SG,Ligue 1,€222.00m,neymar,barcelona
1,17,Ousmane Dembélé,Right Winger,20,33.0,France,Bor. Dortmund,Bundesliga,Barcelona,LaLiga,€135.00m,ousmanedembele,bordortmund
2,17,Virgil van Dijk,Centre-Back,26,30.0,Netherlands,Southampton,Premier League,Liverpool,Premier League,€84.65m,virgilvandijk,southampton
3,17,Álvaro Morata,Centre-Forward,24,40.0,Spain,Real Madrid,LaLiga,Chelsea,Premier League,€66.00m,alvaromorata,realmadrid
4,17,Pierre-Emerick Aubameyang,Centre-Forward,28,65.0,Gabon,Bor. Dortmund,Bundesliga,Arsenal,Premier League,€63.75m,pierreemerickaubameyang,bordortmund
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13970,23,Tomas Suslov,Attacking Midfield,21,2.0,Slovakia,FC Groningen,Keuken Kampioen Divisie,Hellas Verona,Serie A,€1.00m,tomassuslov,fcgroningen
13971,23,Sergi Guardiola,Centre-Forward,32,2.0,Spain,Real Valladolid,2ª División,Cádiz CF,LaLiga,€1.00m,sergiguardiola,realvalladolid
13979,23,Maximilian Mittelstädt,Left-Back,26,2.0,Germany,Hertha BSC,2. Bundesliga,VfB Stuttgart,Bundesliga,€500k,maximilianmittelstadt,herthabsc
13988,23,Maximilian Wittek,Left-Back,27,2.0,Germany,Vitesse,Eredivisie,VfL Bochum,Bundesliga,€750k,maximilianwittek,vitesse


In [148]:
transfers_merged["xGoalDiff"] = transfers_merged["Gls"] - transfers_merged["xG"]

In [149]:
transfers_merged.drop(columns=['ID', 'clubID', "player_id", "short_name", "overall","body_type",
       'stats_name', 'stats_club', 'Liga', 'Abgebender Verein_y', 'Spieler_y',
       'Nationalität_y', 'Position_y', 'Alter_y', 'Geboren', 'born','fifa_name', 'fifa_club_x','long_name',
       'club_name','fifa_club_y','Ablöse','Transfer Typ','MP','Starts','90s','G+A',
       'G-PK','PK','PKatt','npxG','npxG+xAG','Gls p90','Ast p90','G+A p90','G-PK p90','G+A-PK p90','xG p90',
       'xAG p90','xG+xAG p90','npxG p90','npxG+xAG p90','npxG+xAG.1','attacking_crossing','attacking_finishing',
       'attacking_heading_accuracy','attacking_short_passing','attacking_volleys','skill_dribbling','skill_curve',
       'skill_fk_accuracy','skill_long_passing','skill_ball_control','movement_acceleration','movement_sprint_speed','movement_agility','movement_reactions',
       'movement_balance','power_shot_power','power_jumping','power_stamina','power_strength','power_long_shots','mentality_aggression','mentality_interceptions','mentality_positioning',
       'mentality_vision','mentality_penalties','mentality_composure','defending_marking_awareness','defending_standing_tackle','defending_sliding_tackle','goalkeeping_diving','goalkeeping_handling',
       'goalkeeping_kicking','goalkeeping_positioning','goalkeeping_reflexes','goalkeeping_speed'],axis=1,inplace=True)

In [150]:
pd.options.display.max_columns = 111
pd.options.display.max_rows = 100

In [152]:
transfers_merged.rename(columns={"Spieler_x":"Spieler","Position_x":"Position","Alter_x":"Alter","Nationalität_x":"Nationalität",
                                 "Abgebender Verein_x":"Abgebender Verein", "Min":"Minuten", "Gls":"Tore","Ast":"Vorlagen","CrdY":"Gelbe Karten",
                                 "CrdR":"Rote Karten","xG":"Expected Goals","xAG":"Expected Assists","PrgC":"Erfolgreiche Dribblings","PrgP":"Erfolgreiche Pässe",
                                 "potential":"Potential","height_cm":"Größe","weight_kg":"Gewicht","preferred_foot":"Starker Fuß","weak_foot":"Beidfüßigkeit",
                                 "skill_moves":"Skill-Moves","international_reputation":"Internationaler Ruf","work_rate":"Arbeitsrate","pace":"Schnelligkeit",
                                 "passing":"Passen","dribbling":"Dribbling","physic":"Physis","shooting":"Schießen","defending":"Verteidigung"},inplace=True)

In [ ]:
transfers = transfers[transfers['Position'] != 'Goalkeeper']

In [ ]:
def add_transfer_history_column(data):
    data['Vergangene Transfers'] = data.apply(
        lambda row: data[(data['Spieler'] == row['Spieler']) & (data['Saison'] < row['Saison'])].shape[0], axis=1
    )
    return data

transfers = add_transfer_history_column(transfers)

In [ ]:
transfers.isnull().sum().to_frame('nulls')

In [ ]:
# Function to replace missing values with the "Global Closest Fit"
def replace_nulls_with_global_closest_fit(data):
    numeric_columns = data.select_dtypes(include=[np.number]).columns
    for column in numeric_columns:
        if data[column].isnull().any():
            for idx in data[data[column].isnull()].index:
                target_row = data.loc[idx]
                position_filter = data['Position'] == target_row['Position']
                available_data = data[position_filter & data[column].notnull()]

                if not available_data.empty:
                    def compute_distance(row):
                        common_columns = row.dropna().index.intersection(target_row.dropna().index)
                        return distance.euclidean(row[common_columns], target_row[common_columns])
                    
                    distances = available_data[numeric_columns].dropna().apply(compute_distance, axis=1)
                    if not distances.empty:
                        closest_index = distances.idxmin()
                        data.at[idx, column] = data.at[closest_index, column]
    return data


transfers = replace_nulls_with_global_closest_fit(transfers)

In [ ]:
transfers = transfers[transfers['Aufnehmender Verein'] != 'Al Ahly']
transfers = transfers[transfers['Aufnehmender Verein'] != 'Zamalek']
transfers = transfers[transfers['Aufnehmender Verein'] != 'Wolfsberger AC']
transfers = transfers[transfers['Aufnehmender Verein'] != 'LASK']
transfers = transfers[transfers['Aufnehmender Verein'] != 'Sturm Graz']
transfers = transfers[transfers['Aufnehmender Verein'] != 'RB Salzburg']

In [154]:
transfers_merged.to_csv("C:\\Users\\soere\\Bachelorarbeit\\datasets\\transfers_dataset_merged.csv",index=False)